In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from environment import Environment, DirectedTree, DEFAULT, CONSTITUENT
from prouter import Router
from sampler import Sampler, STRATEGIES
import logging
import json
import os
from tqdm import tqdm

c:\users\albert\projekte\swmmpulse\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def preparation_env():
    env = Environment()
    #env.read_swmmoutfile(r"C:\Users\albert/Documents/SWMMpulse/HS_calib_120_simp.out")
    env.read_swmmoutfile(r"demo_swmm_out.out")
    #graph = DirectedTree.from_swmm(r"C:\Users\albert/Documents/SWMMpulse/HS_calib_120_simp.inp")
    graph = DirectedTree.from_swmm(r"demo_swmm-inp.inp")
    #node_data = pd.read_csv(r"C:\Users\albert/Documents/SWMMpulse/HS_calib_120_simp/pop_node_data.csv")
    node_data = pd.read_csv(r"pop_node_data.csv")
    node_data = node_data.set_index("NAME").to_dict(orient="index")
    graph.add_nodevalues(node_data)
    env.add_graph(graph)
    return env

def set_infectivity(r_inf, env):
    env.information["groups"][0]["weight"] = 1 - r_inf
    env.information["groups"][1]["weight"] = r_inf
    return env

## Simulations with fixed infection rates
n Simulations for a given infection rate scenario are simulated and routetables and timeseries are stored at a given location.
A set of infection rate scenarios is run

In [6]:
path = r"C:\Users\albert\Documents\SWMMpulse"
runnr = 1
run = f"00_run{runnr-1:d}"

router = Router()
env = preparation_env()
groot = env.graph.root

cov = env.information["constituents"]["Cov-RNA"]
cov["fractions"] = [1]
cov["skewedness"] = [[1,1]]
env.information["constituents"]["Cov-RNA"] = cov

bins = pd.interval_range(start=0, end=7200, freq=1200)
bins = bins.append(pd.interval_range(start=7200, end=14400, freq=3600))
n = 250

simid = 0

for r_inf in np.arange(0.002, 0.016, 0.002):
    
    #update environment with new infectivity
    env = set_infectivity(r_inf, env)
    router.add_environment(env)
    
    for i in tqdm(range(n), desc=f"infectionrate {r_inf:.3f}"):
        #generate packets
        packets = router.environment.get_packets()
        fname = f"HS120_{f'{r_inf:.3f}'[2:]}_packets_{simid:06d}.json"
        packets.to_json(os.path.join(path, "00_packets", run, fname))
        
        #generate routetable
        routetable = router.route(packets=packets)
        
        #write routetable at network root to file
        fname = f"HS120_{f'{r_inf:.3f}'[2:]}_routetable_{simid:06d}.json"
        with open(os.path.join(path, "01_routetables", run, fname),"w") as jobj:
            jobj.write(json.dumps(routetable[groot]))
        
        #calculate timeseries data at network root
        processed = router.postprocess(routetable, packets, env.information["constituents"].get(CONSTITUENT.COV))
        fname = f"HS120_{f'{r_inf:.3f}'[2:]}_processed_{simid:06d}.json"
        
        #create dataframe
        timeseries = pd.DataFrame(processed, index=pd.date_range("2000-01-01", freq="10S", periods=8640))
        #calculate timeseries for entire catchment
        timeseries.sum(axis=1).to_json(os.path.join(path, "02_processed", run, fname))
        
        simid += 1

print("----------------------- MonteCarlo runs finished ----------------------------------")

infectionrate 0.014: 100%|███████████████████████████████████████████████████████████| 250/250 [16:44<00:00,  4.02s/it]

----------------------- MonteCarlo runs finished ----------------------------------


## Simulations with moving infectin rates
n Simulations are run for infection rate scenarios between a start rate end an end rate. in between values are interpolated linearly by simulation id.

In [4]:
path = r"C:\Users\albert\Documents\SWMMpulse"
runnr = 2
run = f"00_run{runnr-1:d}"

router = Router()
env = preparation_env()
groot = env.graph.root

cov = env.information["constituents"]["Cov-RNA"]
cov["fractions"] = [1]
cov["skewedness"] = [[1,1]]
env.information["constituents"]["Cov-RNA"] = cov

bins = pd.interval_range(start=0, end=7200, freq=1200)
bins = bins.append(pd.interval_range(start=7200, end=14400, freq=3600))
n = 2000

simid = 0

for i in tqdm(range(n), desc="Running scenarios..."):
    
    r_inf = np.around(0.002 + i*(0.016 - 0.002)/n, 4)
    
    #update environment with new infectivity
    env = set_infectivity(r_inf, env)
    router.add_environment(env)
    
    #generate packets
    packets = router.environment.get_packets()
    fname = f"HS120_{f'{r_inf:.3f}'[2:]}_packets_{simid:06d}.json"
    packets.to_json(os.path.join(path, "00_packets", run, fname))

    #generate routetable
    routetable = router.route(packets=packets)

    #write routetable at network root to file
    fname = f"HS120_{f'{r_inf:.3f}'[2:]}_routetable_{simid:06d}.json"
    with open(os.path.join(path, r"01_routetables", run, fname),"w") as jobj:
        jobj.write(json.dumps(routetable[groot]))

    #calculate timeseries data at network root
    processed = router.postprocess(routetable, packets, env.information["constituents"].get(CONSTITUENT.COV))
    fname = f"HS120_{f'{r_inf:.3f}'[2:]}_processed_{simid:06d}.json"

    #create dataframe
    timeseries = pd.DataFrame(processed, index=pd.date_range("2000-01-01", freq="10S", periods=8640))
    #calculate timeseries for entire catchment
    timeseries.sum(axis=1).to_json(os.path.join(path, "02_processed", run, fname))
        
    simid += 1
    
print("----------------------- MonteCarlo runs finished ----------------------------------")

Running scenarios...: 100%|██████████████████████████████████████████████████████| 2000/2000 [1:57:01<00:00,  3.51s/it]

----------------------- MonteCarlo runs finished ----------------------------------


{'name': 'Cov-RNA',
 'specific_load': 1000,
 'unit': '#',
 'decay_rate': 0.114,
 'groups': ['infected'],
 'fractions': [1],
 'skewedness': [1, 1]}